# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

FileNotFoundError: [Errno 2] No such file or directory: 'data/us_job_market_discrimination.dta'

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [8]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.shape

(4870, 65)

In [14]:
df= data[['race','call']]

In [18]:
df_call = df[df['call']==1.0]

In [19]:
df_call.groupby('race').count()

,call
race,
b,157
w,235


** 1.What test is appropriate for this problem? Does CLT apply? **

We can use 2 population permutation test.

Yes CLT do apply

** 2.What are the null and alternate hypotheses? **

Null hypotheses is there is no racial descrimination in United States Labor Market.

Alternate hypotheses is there is racial descrimination in United States Labor Market.

** 3.Compute margin of error, confidence interval, and p-value. **

In [26]:
df_b = df[df['race']=='b']
df_w = df[df['race']=='w']

In [54]:
df_b.shape == df_w.shape

True

In [57]:
b_call_sum = np.sum(df_b['call'])
b_call_sum

157.0

In [60]:
def permuted_sample(data1,data2):
    data = np.concatenate((data1,data2))
    perm_data = np.random.permutation(data)
    perm_data1 = perm_data[:len(data1)]
    perm_data2 = perm_data[len(data1):]
    return perm_data1,perm_data2

In [61]:
def diff_of_sum(data1,data2):
    return np.sum(data1) - np.sum(data2)

In [62]:
def perm_rep_func(data1,data2,func):
    perm_rep = np.empty(10000)
    for i in range(10000):
        perm_sample_w, perm_sample_b = permuted_sample(data1,data2)
        perm_rep[i] = func(perm_sample_w, perm_sample_b)
    return  perm_rep 

In [63]:
emp_diff_sum = np.sum(df_w['call']) - np.sum(df_b['call'])
emp_diff_sum

78.0

In [64]:
array_diff_sum = perm_rep_func(df_w['call'],df_b['call'],diff_of_sum)
p = np.sum(array_diff_sum >= emp_diff_sum)/len(array_diff_sum)
p

0.0

** p value is 0 which means the Null hypothesis is completely true there is bias between black & white candidate when selecting the resume for interview **

In [68]:
def perm_sum_conf(data1,data2):
    perm_b = np.empty(10000)
    perm_w = np.empty(10000)
    for i in range(10000):
        perm_sample_w, perm_sample_b = permuted_sample(data1,data2)
        perm_b[i] = np.sum(perm_sample_b)
        perm_w[i] = np.sum(perm_sample_w)
    return  perm_b,perm_w 

In [69]:
sum_b_call, sum_w_call = perm_sum_conf(df_b['call'],df_w['call'])

In [70]:
def draw_bs_pairs_linreg(x, y, size=1):
    """Perform pairs bootstrap for linear regression."""

    # Set up array of indices to sample from: inds
    inds = np.arange(len(x))

    # Initialize replicates: bs_slope_reps, bs_intercept_reps
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_inds = np.random.choice(inds, size=len(inds))
        bs_x, bs_y = x[bs_inds], y[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = np.polyfit(bs_x,bs_y,1)

    return bs_slope_reps, bs_intercept_reps

In [71]:
bs_slope_reps, bs_intercept_reps = draw_bs_pairs_linreg(sum_b_call,sum_w_call,10000)
print(np.percentile(bs_slope_reps, [2.5,97.5]))

[-1. -1.]


** There is 95% confidence interval **

** 4.Write a story describing the statistical significance in the context or the original problem.**

We concluded that statistically there are no indication of racial discrimination in United States Labor Market. The p values indicates the NULL hypothesis is 100% true.

** 5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis? **

My analysis indicates that race/name is not the most important factor in callback success because the p value indicates that.